In [2]:
# ================================================================
# 모듈 7: 종합평가 점수 산정 모듈 + CL별 정규화 (연말 팀 단위 처리) - 수정본
# ================================================================

from typing import Annotated, List, Literal, TypedDict, Dict, Optional
from langchain_core.messages import HumanMessage 
import operator
from langgraph.graph import StateGraph, START, END
import json
import re
import sys
import os
import statistics

# 기존 imports
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Row
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, AIMessage
from dotenv import load_dotenv

load_dotenv()

# DB 설정
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../../..')))
from config.settings import DatabaseConfig

db_config = DatabaseConfig()
DATABASE_URL = db_config.DATABASE_URL
engine = create_engine(DATABASE_URL, pool_pre_ping=True)

# LLM 클라이언트 설정
llm_client = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print(f"LLM Client initialized: {llm_client.model_name}")

def row_to_dict(row: Row) -> Dict:
    """SQLAlchemy Row 객체를 딕셔너리로 변환"""
    if row is None:
        return {}
    return row._asdict()

def _extract_json_from_llm_response(text: str) -> str:
    """LLM 응답에서 JSON 블록 추출"""
    match = re.search(r"```(?:json)?\s*(.*?)\s*```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

# ================================================================
# TeamModule7AgentState 정의 - 팀 단위 처리
# ================================================================

class TeamModule7AgentState(TypedDict):
    """모듈 7 (종합평가 점수 산정) 상태 - 팀 단위 처리"""
    messages: Annotated[List[HumanMessage], operator.add]
    
    # 팀 기본 정보
    team_id: str
    period_id: int  # 연말: 4
    
    # 팀 전체 데이터 (한 번에 조회)
    team_members: List[Dict]  # 팀원 기본 정보
    team_achievement_data: List[Dict]  # 팀 전체 달성률 데이터
    team_fourp_data: List[Dict]  # 팀 전체 4P 데이터
    team_quarterly_data: Dict  # 팀원별 분기 데이터
    
    # 통계 계산 결과 (팀 공통)
    achievement_stats: Dict  # 평균, 표준편차 등
    weights_by_cl: Dict  # CL별 가중치
    
    # 개별 계산 결과
    individual_scores: List[Dict]  # 각 팀원별 점수
    evaluation_comments: List[Dict]  # 각 팀원별 코멘트
    
    # 처리 결과
    processed_count: int
    failed_members: List[str]

# ================================================================
# 가중치 계산 함수
# ================================================================

def get_evaluation_weights_by_cl(cl) -> Dict[str, float]:
    """CL별 디폴트 가중치 반환"""
    # CL 값을 문자열로 변환 (숫자든 문자열이든 처리)
    if isinstance(cl, (int, float)):
        cl_key = f"CL{int(cl)}"
    else:
        cl_key = str(cl).upper()
        if not cl_key.startswith("CL"):
            cl_key = f"CL{cl_key}"
    
    cl_weights = {
        "CL3": {"achievement": 0.6, "fourp": 0.4},
        "CL2": {"achievement": 0.5, "fourp": 0.5}, 
        "CL1": {"achievement": 0.4, "fourp": 0.6}
    }
    return cl_weights.get(cl_key, {"achievement": 0.5, "fourp": 0.5})  # 기본값

# ================================================================
# CL별 정규화 관련 함수들
# ================================================================

def get_cl_normalization_params(cl) -> Dict[str, float]:
    """CL별 정규화 파라미터 반환 (SK 표준)"""
    # CL 값을 문자열로 변환 (숫자든 문자열이든 처리)
    if isinstance(cl, (int, float)):
        cl_key = f"CL{int(cl)}"
    else:
        cl_key = str(cl).upper()
        if not cl_key.startswith("CL"):
            cl_key = f"CL{cl_key}"
    
    params = {
        "CL3": {"target_mean": 3.5, "target_stdev": 1.7},
        "CL2": {"target_mean": 3.5, "target_stdev": 1.5}, 
        "CL1": {"target_mean": 3.5, "target_stdev": 1.4}
    }
    return params.get(cl_key, {"target_mean": 3.5, "target_stdev": 1.5})

def normalize_cl_group(members: List[Dict], cl: str) -> List[Dict]:
    """CL 그룹 내 정규화 실행 (4명 이상일 때만 적용)"""
    
    if len(members) == 0:
        return members
    
    print(f"   {cl} 그룹 ({len(members)}명) 정규화 처리:")
    
    # 3명 이하인 경우 원시점수 유지
    if len(members) <= 3:
        for member in members:
            member["normalized_score"] = member["hybrid_score"]  # 원시점수 그대로
            member["normalization_reason"] = f"팀 내 {cl} {len(members)}명 (원시점수 유지)"
            member["raw_hybrid_score"] = member["hybrid_score"]  # 원시점수 보관
            print(f"     {member['emp_no']}: {member['hybrid_score']:.2f}점 (원시점수 유지)")
        return members
    
    # 4명 이상인 경우 정규화 적용
    # CL별 목표 파라미터
    params = get_cl_normalization_params(cl)
    target_mean = params["target_mean"]
    target_stdev = params["target_stdev"]
    
    # 원시점수 수집 (하이브리드 점수)
    raw_scores = [m["hybrid_score"] for m in members]
    
    # 현재 통계
    current_mean = statistics.mean(raw_scores)
    current_stdev = statistics.stdev(raw_scores)
    
    print(f"     정규화 적용: 평균 {current_mean:.2f} → {target_mean}, 표준편차 {current_stdev:.2f} → {target_stdev}")
    
    # 정규화 적용
    for member in members:
        raw_score = member["hybrid_score"]
        
        if current_stdev == 0:
            # 모든 점수가 동일한 경우
            normalized_score = target_mean
            reason = f"{cl} 동일점수 → 평균 {target_mean}점"
        else:
            # Z-score 계산 후 목표 분포로 변환
            z_score = (raw_score - current_mean) / current_stdev
            normalized_score = target_mean + (z_score * target_stdev)
            
            # 0.0-5.0 범위 제한 (SK 기준)
            normalized_score = max(0.0, min(5.0, normalized_score))
            
            reason = f"{cl} 정규화 (Z-Score: {z_score:.2f})"
        
        member["normalized_score"] = round(normalized_score, 2)
        member["normalization_reason"] = reason
        member["raw_hybrid_score"] = raw_score  # 원시점수 보관
        
        print(f"     {member['emp_no']}: {raw_score:.2f} → {normalized_score:.2f} ({reason})")
    
    return members

# ================================================================
# 팀 단위 DB 조회 함수들
# ================================================================

def fetch_team_members(team_id: str) -> List[Dict]:
    """팀원 기본 정보 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT emp_no, emp_name, cl, position, team_id
            FROM employees 
            WHERE team_id = :team_id
            ORDER BY cl DESC, position DESC
        """)
        results = connection.execute(query, {"team_id": team_id}).fetchall()
        return [row_to_dict(row) for row in results]

def fetch_team_achievement_data(team_id: str, period_id: int) -> List[Dict]:
    """팀 전체 달성률 데이터 한 번에 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT fer.emp_no, fer.contribution_rate, 
                   fer.ai_annual_achievement_rate,
                   fer.ai_annual_performance_summary_comment, 
                   fer.ai_peer_talk_summary,
                   e.emp_name, e.cl, e.position
            FROM final_evaluation_reports fer
            JOIN team_evaluations te ON fer.team_evaluation_id = te.team_evaluation_id
            JOIN employees e ON fer.emp_no = e.emp_no
            WHERE e.team_id = :team_id AND te.period_id = :period_id
            ORDER BY fer.ai_annual_achievement_rate DESC
        """)
        results = connection.execute(query, {"team_id": team_id, "period_id": period_id}).fetchall()
        return [row_to_dict(row) for row in results]

def fetch_team_fourp_data(team_id: str, period_id: int) -> List[Dict]:
    """팀 전체 4P 데이터 한 번에 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT fer.emp_no, fer.ai_4p_evaluation, e.emp_name
            FROM final_evaluation_reports fer
            JOIN team_evaluations te ON fer.team_evaluation_id = te.team_evaluation_id
            JOIN employees e ON fer.emp_no = e.emp_no
            WHERE e.team_id = :team_id AND te.period_id = :period_id
        """)
        results = connection.execute(query, {"team_id": team_id, "period_id": period_id}).fetchall()
        
        fourp_data = []
        for row in results:
            try:
                fourp_evaluation = json.loads(row.ai_4p_evaluation) if row.ai_4p_evaluation else {}
                fourp_data.append({
                    "emp_no": row.emp_no,
                    "emp_name": row.emp_name,
                    "fourp_results": fourp_evaluation
                })
            except json.JSONDecodeError:
                print(f"4P JSON 파싱 실패: {row.emp_no}")
                fourp_data.append({
                    "emp_no": row.emp_no,
                    "emp_name": row.emp_name,
                    "fourp_results": {}
                })
        
        return fourp_data

def fetch_team_quarterly_data(team_id: str, period_id: int) -> Dict:
    """팀 전체 분기별 Task 데이터 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT t.emp_no, ts.period_id, ts.task_id, t.task_name,
                   ts.ai_contribution_score, ts.ai_analysis_comment_task, ts.task_performance
            FROM task_summaries ts
            JOIN tasks t ON ts.task_id = t.task_id
            JOIN employees e ON t.emp_no = e.emp_no
            WHERE e.team_id = :team_id AND ts.period_id <= :period_id
            ORDER BY t.emp_no, ts.period_id, ts.task_id
        """)
        results = connection.execute(query, {"team_id": team_id, "period_id": period_id}).fetchall()
        
        # emp_no별로 그룹화
        quarterly_data = {}
        for row in results:
            emp_no = row.emp_no
            if emp_no not in quarterly_data:
                quarterly_data[emp_no] = []
            quarterly_data[emp_no].append(row_to_dict(row))
        
        return quarterly_data

def batch_update_temp_evaluations(score_data: List[Dict]) -> Dict:
    """팀 전체 temp_evaluations 배치 업데이트 (raw_score, score 모두 저장)"""
    success_count = 0
    failed_members = []
    
    with engine.connect() as connection:
        try:
            for data in score_data:
                try:
                    query = text("""
                        UPDATE temp_evaluations 
                        SET ai_reason = :ai_reason,
                            raw_score = :raw_score,
                            score = :score,
                            comment = :comment
                        WHERE TempEvaluation_empNo = :emp_no
                    """)
                    
                    result = connection.execute(query, {
                        "emp_no": data["emp_no"],
                        "ai_reason": data["ai_reason"],
                        "raw_score": data["raw_score"],  # 원시점수 저장
                        "score": data["score"],  # 정규화된 점수 저장
                        "comment": data["comment"]
                    })
                    
                    if result.rowcount > 0:
                        success_count += 1
                        print(f"DB 업데이트 성공: {data['emp_no']} (원시: {data['raw_score']}, 정규화: {data['score']})")
                    else:
                        failed_members.append(data["emp_no"])
                        print(f"DB 업데이트 실패: {data['emp_no']} (행 없음)")
                        
                except Exception as e:
                    failed_members.append(data["emp_no"])
                    print(f"DB 업데이트 실패: {data['emp_no']} - {e}")
            
            connection.commit()
            print(f"배치 업데이트 완료: 성공 {success_count}건, 실패 {len(failed_members)}건")
            
            return {
                "success_count": success_count,
                "failed_members": failed_members
            }
            
        except Exception as e:
            print(f"배치 업데이트 실패: {e}")
            connection.rollback()
            return {
                "success_count": 0,
                "failed_members": [data["emp_no"] for data in score_data]
            }

# ================================================================
# Z-Score 달성률 점수 계산 함수
# ================================================================

def calculate_team_achievement_stats(achievement_data: List[Dict]) -> Dict:
    """팀 달성률 통계 계산 (Z-Score용)"""
    achievement_rates = [data["ai_annual_achievement_rate"] for data in achievement_data if data.get("ai_annual_achievement_rate") is not None]
    
    if len(achievement_rates) <= 1:
        return {
            "mean": achievement_rates[0] if achievement_rates else 0,
            "stdev": 0,
            "count": len(achievement_rates),
            "min": min(achievement_rates) if achievement_rates else 0,
            "max": max(achievement_rates) if achievement_rates else 0
        }
    
    return {
        "mean": statistics.mean(achievement_rates),
        "stdev": statistics.stdev(achievement_rates),
        "count": len(achievement_rates),
        "min": min(achievement_rates),
        "max": max(achievement_rates)
    }

def calculate_zscore_achievement_score(achievement_rate: float, team_stats: Dict) -> tuple[float, str]:
    """Z-Score 방식으로 달성률 점수 계산"""
    
    if team_stats["count"] <= 1:
        return 3.5, "팀원 1명 (기본 3.5점)"
    
    if team_stats["stdev"] == 0:
        return 3.5, f"팀 내 달성률 동일 ({achievement_rate:.1f}%, 기본 3.5점)"
    
    # Z-Score 계산
    z_score = (achievement_rate - team_stats["mean"]) / team_stats["stdev"]
    
    # Z-Score를 1-5점으로 변환 (평균 3.5점, 표준편차 1당 ±0.8점)
    base_score = 3.5
    score_range = 0.8
    calculated_score = base_score + (z_score * score_range)
    
    # 1-5점 범위로 제한
    final_score = max(1.0, min(5.0, calculated_score))
    
    # 성과 레벨 판정
    if z_score > 1.5:
        performance_level = "매우 우수"
    elif z_score > 0.5:
        performance_level = "우수"
    elif z_score > -0.5:
        performance_level = "평균"
    elif z_score > -1.5:
        performance_level = "개선 필요"
    else:
        performance_level = "크게 개선 필요"
    
    reason = f"팀 평균 {team_stats['mean']:.1f}% 대비 Z-Score {z_score:.2f} ({performance_level})"
    
    return round(final_score, 2), reason

# ================================================================
# LLM 호출 함수
# ================================================================

def call_llm_for_normalized_evaluation_comments(
    emp_data: Dict,
    normalized_score: float,
    raw_hybrid_score: float,
    achievement_score: float,
    fourp_score: float,
    quarterly_tasks: List[Dict],
    fourp_results: Dict,
    achievement_reason: str,
    normalization_reason: str
) -> Dict:
    """정규화된 점수로 ai_reason과 comment 생성"""
    
    emp_no = emp_data["emp_no"]
    emp_name = emp_data.get("emp_name", emp_no)
    position = emp_data.get("position", "직책 정보 없음")
    cl = emp_data.get("cl", "CL 정보 없음")
    
    print(f"LLM Call: {emp_no} 정규화 후 종합평가 근거 생성")
    
    # 분기별 Task 요약
    quarterly_summary = ""
    for i, task in enumerate(quarterly_tasks[:8]):  # 최대 8개만 표시
        quarterly_summary += f"Q{task.get('period_id')}: {task.get('task_name')} - 달성률 {task.get('ai_contribution_score', 0)}점\n"
        if task.get('ai_analysis_comment_task'):
            quarterly_summary += f"  → {task.get('ai_analysis_comment_task')}\n"
    
    if not quarterly_summary:
        quarterly_summary = "분기별 Task 데이터 없음"
    
    # 4P 요약
    fourp_summary = f"""
    - Passionate: {fourp_results.get('passionate', {}).get('score', 3.0)}점
    - Proactive: {fourp_results.get('proactive', {}).get('score', 3.0)}점  
    - Professional: {fourp_results.get('professional', {}).get('score', 3.0)}점
    - People: {fourp_results.get('people', {}).get('score', 3.0)}점
    - 평균: {fourp_results.get('overall', {}).get('average_score', 3.0)}점
    """
    
    system_prompt = """
    당신은 SK 조직의 종합 평가 전문가입니다.
    정량평가(달성률)와 정성평가(4P BARS)를 종합하고 CL별 정규화를 거친 최종 점수를 바탕으로 
    두 가지 관점의 평가 근거를 생성해주세요.

    평가 기준 (SK 표준):
    - 5.0점: 탁월한 성과 (상위 5%)
    - 4.0-4.9점: 우수한 성과 (상위 20%)  
    - 3.5점: 기대 수준 (평균)
    - 3.0-3.4점: 기대 부합 (중간 60%)
    - 2.0-2.9점: 개선 필요 (하위 15%)

    두 가지 관점의 근거를 생성하세요:
    1. ai_reason: 팀장이 보는 AI 평가 근거 (객관적, 분석적, CL별 정규화 과정 포함)
    2. comment: 팀원이 보는 평가 근거 초안 (개인 친화적, 발전지향적, 격려 포함)

    결과는 JSON 형식으로만 응답하세요.
    """
    
    human_prompt = f"""
    <직원 정보>
    이름: {emp_name}
    사번: {emp_no}
    직책: {position}
    CL: {cl}
    </직원 정보>

    <점수 산출 과정>
    최종 정규화 점수: {normalized_score}점
    ├── 원시 하이브리드 점수: {raw_hybrid_score:.2f}점
    │   ├── 달성률 점수: {achievement_score}점 ({achievement_reason})
    │   └── 4P 점수: {fourp_score}점
    └── CL별 정규화: {normalization_reason}
    </점수 산출 과정>

    <상세 근거 데이터>
    연간 달성률: {emp_data.get('ai_annual_achievement_rate', 0)}%
    성과 요약: {emp_data.get('ai_annual_performance_summary_comment', '성과 요약 없음')}
    동료평가: {emp_data.get('ai_peer_talk_summary', '동료평가 없음')}
    
    4P 평가 결과:
    {fourp_summary}
    
    분기별 Task 성과:
    {quarterly_summary}
    </상세 근거 데이터>

    JSON 응답:
    {{
        "ai_reason": "[팀장용: {emp_name}({emp_no})님에 대한 객관적이고 구체적인 AI 평가 근거. 팀 내 Z-Score 결과와 CL별 정규화 과정을 포함하여 분석적 관점으로 설명]",
        "comment": "[팀원용: {emp_name}님께 드리는 개인 친화적이고 발전지향적인 평가 근거 초안. 최종 점수 {normalized_score}점에 대한 격려와 성장 방향 포함]"
    }}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_prompt),
        HumanMessage(content=human_prompt)
    ])
    
    chain = prompt | llm_client

    try:
        response: AIMessage = chain.invoke({})
        json_output_raw = response.content
        json_output = _extract_json_from_llm_response(json_output_raw)
        llm_parsed_data = json.loads(json_output)
        
        ai_reason = llm_parsed_data.get("ai_reason", "")
        comment = llm_parsed_data.get("comment", "")

        if not ai_reason or not comment:
            raise ValueError("LLM 응답에서 ai_reason 또는 comment가 누락됨")

        return {
            "ai_reason": ai_reason,
            "comment": comment
        }

    except json.JSONDecodeError as e:
        print(f"LLM 응답 JSON 파싱 오류: {e}")
        return {
            "ai_reason": f"{emp_name}님의 정규화 후 종합 평가 근거 생성 중 오류 발생",
            "comment": f"{emp_name}님께 드리는 정규화 후 평가 근거 생성 중 오류 발생"
        }
    except Exception as e:
        print(f"LLM 호출 중 오류: {e}")
        return {
            "ai_reason": f"{emp_name}님의 정규화 후 종합 평가 근거 생성 중 오류 발생",
            "comment": f"{emp_name}님께 드리는 정규화 후 평가 근거 생성 중 오류 발생"
        }

# ================================================================
# 팀 단위 서브모듈 함수들
# ================================================================

def team_data_collection_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """1. 팀 데이터 수집 서브모듈"""
    
    team_id = state["team_id"]
    period_id = state["period_id"]
    
    try:
        print(f"🔍 팀 데이터 수집 시작: {team_id}")
        
        # 1. 팀원 기본 정보 조회
        team_members = fetch_team_members(team_id)
        print(f"   팀원 수: {len(team_members)}명")
        
        # 2. 팀 전체 달성률 데이터 조회
        team_achievement_data = fetch_team_achievement_data(team_id, period_id)
        print(f"   달성률 데이터: {len(team_achievement_data)}건")
        
        # 3. 팀 전체 4P 데이터 조회
        team_fourp_data = fetch_team_fourp_data(team_id, period_id)
        print(f"   4P 데이터: {len(team_fourp_data)}건")
        
        # 4. 팀 전체 분기별 데이터 조회
        team_quarterly_data = fetch_team_quarterly_data(team_id, period_id)
        print(f"   분기별 데이터: {len(team_quarterly_data)}명")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"데이터 수집 완료")],
            "team_members": team_members,
            "team_achievement_data": team_achievement_data,
            "team_fourp_data": team_fourp_data,
            "team_quarterly_data": team_quarterly_data
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"데이터 수집 실패: {str(e)}")]
        raise e

def team_statistics_calculation_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """2. 팀 통계 계산 서브모듈"""
    
    try:
        team_achievement_data = state["team_achievement_data"]
        team_members = state["team_members"]
        
        # 달성률 통계 계산
        achievement_stats = calculate_team_achievement_stats(team_achievement_data)
        print(f"📊 달성률 통계: 평균 {achievement_stats['mean']:.1f}%, 표준편차 {achievement_stats['stdev']:.1f}%")
        
        # CL별 가중치 계산
        weights_by_cl = {}
        for member in team_members:
            cl = member.get("cl", "CL2")
            if cl not in weights_by_cl:
                weights_by_cl[cl] = get_evaluation_weights_by_cl(cl)
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content="통계 계산 완료")],
            "achievement_stats": achievement_stats,
            "weights_by_cl": weights_by_cl
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"통계 계산 실패: {str(e)}")]
        raise e

def team_score_calculation_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """3. 팀 전체 점수 계산 서브모듈 (하이브리드 원시점수)"""
    
    try:
        team_achievement_data = state["team_achievement_data"]
        team_fourp_data = state["team_fourp_data"]
        achievement_stats = state["achievement_stats"]
        weights_by_cl = state["weights_by_cl"]
        
        individual_scores = []
        
        print("🧮 개별 하이브리드 점수 계산 시작...")
        
        for achievement_data in team_achievement_data:
            emp_no = achievement_data["emp_no"]
            cl = achievement_data.get("cl", "CL2")
            
            # 가중치 조회
            weights = weights_by_cl.get(cl, {"achievement": 0.5, "fourp": 0.5})
            
            # Z-Score 달성률 점수 계산
            achievement_score, achievement_reason = calculate_zscore_achievement_score(
                achievement_data["ai_annual_achievement_rate"], 
                achievement_stats
            )
            
            # 4P 점수 조회
            fourp_data = next((fp for fp in team_fourp_data if fp["emp_no"] == emp_no), {})
            fourp_results = fourp_data.get("fourp_results", {})
            fourp_score = fourp_results.get("overall", {}).get("average_score", 3.0)
            
            # 하이브리드 점수 계산 (정규화 전 원시점수)
            hybrid_score = (achievement_score * weights["achievement"]) + (fourp_score * weights["fourp"])
            final_score = round(hybrid_score, 2)
            
            individual_scores.append({
                "emp_no": emp_no,
                "emp_name": achievement_data.get("emp_name"),
                "cl": cl,
                "achievement_score": achievement_score,
                "achievement_reason": achievement_reason,
                "fourp_score": fourp_score,
                "hybrid_score": final_score,  # 정규화 전 원시점수
                "weights": weights,
                "emp_data": achievement_data,
                "fourp_results": fourp_results
            })
            
            print(f"   {emp_no}: {final_score}점 (달성률 {achievement_score}, 4P {fourp_score})")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"하이브리드 점수 계산 완료: {len(individual_scores)}명")],
            "individual_scores": individual_scores
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"점수 계산 실패: {str(e)}")]
        raise e

def team_normalization_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """4. 팀 내 CL별 정규화 서브모듈"""
    
    try:
        individual_scores = state["individual_scores"]
        
        print("🔄 팀 내 CL별 정규화 시작...")
        
        # 1. CL별 그룹화 (숫자/문자열 모두 처리)
        cl_groups = {
            "CL1": [],
            "CL2": [], 
            "CL3": []
        }
        
        for score in individual_scores:
            cl_raw = score.get("cl", 2)  # 기본값 2
            
            # CL 값 정규화
            if isinstance(cl_raw, (int, float)):
                cl = f"CL{int(cl_raw)}"
            else:
                cl = str(cl_raw).upper()
                if not cl.startswith("CL"):
                    cl = f"CL{cl}"
            
            # 유효한 CL인지 확인
            if cl in cl_groups:
                cl_groups[cl].append(score)
                score["cl"] = cl  # 정규화된 CL 값으로 업데이트
            else:
                print(f"⚠️ 알 수 없는 CL: {cl_raw} → CL2로 처리")
                cl_groups["CL2"].append(score)
                score["cl"] = "CL2"
        
        print(f"   CL별 분포: CL3({len(cl_groups['CL3'])}명), CL2({len(cl_groups['CL2'])}명), CL1({len(cl_groups['CL1'])}명)")
        
        # 2. CL별 정규화 실행 (4명 이상일 때만)
        normalized_scores = []
        
        for cl, members in cl_groups.items():
            if len(members) > 0:
                print(f"\n📊 {cl} 정규화 처리:")
                normalized_members = normalize_cl_group(members, cl)
                normalized_scores.extend(normalized_members)
        
        # 3. 정규화 통계 출력
        raw_scores = [s["hybrid_score"] for s in individual_scores]
        norm_scores = [s["normalized_score"] for s in normalized_scores]
        
        print(f"\n📈 정규화 결과:")
        print(f"   원시점수: 평균 {statistics.mean(raw_scores):.2f}, 표준편차 {statistics.stdev(raw_scores) if len(raw_scores) > 1 else 0:.2f}")
        print(f"   정규화점수: 평균 {statistics.mean(norm_scores):.2f}, 표준편차 {statistics.stdev(norm_scores) if len(norm_scores) > 1 else 0:.2f}")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"CL별 정규화 완료: {len(normalized_scores)}명")],
            "individual_scores": normalized_scores
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"정규화 실패: {str(e)}")]
        raise e

def team_comment_generation_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """5. 팀 전체 코멘트 생성 서브모듈 (정규화된 점수 기준)"""
    
    try:
        individual_scores = state["individual_scores"]
        team_quarterly_data = state["team_quarterly_data"]
        
        evaluation_comments = []
        
        print("💬 정규화 후 평가 코멘트 생성 시작...")
        
        for score_data in individual_scores:
            emp_no = score_data["emp_no"]
            quarterly_tasks = team_quarterly_data.get(emp_no, [])
            
            # LLM을 통한 정규화 후 코멘트 생성
            llm_result = call_llm_for_normalized_evaluation_comments(
                emp_data=score_data["emp_data"],
                normalized_score=score_data["normalized_score"],
                raw_hybrid_score=score_data["raw_hybrid_score"],
                achievement_score=score_data["achievement_score"],
                fourp_score=score_data["fourp_score"],
                quarterly_tasks=quarterly_tasks,
                fourp_results=score_data["fourp_results"],
                achievement_reason=score_data["achievement_reason"],
                normalization_reason=score_data["normalization_reason"]
            )
            
            evaluation_comments.append({
                "emp_no": emp_no,
                "emp_name": score_data.get("emp_name"),
                "raw_score": score_data["raw_hybrid_score"],  # 원시점수
                "score": score_data["normalized_score"],  # 정규화된 점수
                "ai_reason": llm_result["ai_reason"],
                "comment": llm_result["comment"]
            })
            
            print(f"   {emp_no}: 정규화 후 코멘트 생성 완료")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"코멘트 생성 완료: {len(evaluation_comments)}명")],
            "evaluation_comments": evaluation_comments
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"코멘트 생성 실패: {str(e)}")]
        raise e

def team_batch_storage_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """6. 팀 배치 저장 서브모듈"""
    
    try:
        evaluation_comments = state["evaluation_comments"]
        
        print("💾 배치 저장 시작...")
        
        # 배치 업데이트 실행
        update_result = batch_update_temp_evaluations(evaluation_comments)
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"배치 저장 완료: 성공 {update_result['success_count']}건")],
            "processed_count": update_result["success_count"],
            "failed_members": update_result["failed_members"]
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"배치 저장 실패: {str(e)}")],
            "processed_count": 0,
            "failed_members": []
        })
        raise e

# ================================================================
# 팀 단위 워크플로우 생성
# ================================================================

def create_team_module7_graph():
    """팀 단위 모듈 7 그래프 생성 및 반환 (CL 정규화 포함)"""
    team_module7_workflow = StateGraph(TeamModule7AgentState)
    
    # 노드 추가
    team_module7_workflow.add_node("team_data_collection", team_data_collection_submodule)
    team_module7_workflow.add_node("team_statistics", team_statistics_calculation_submodule)
    team_module7_workflow.add_node("team_score_calculation", team_score_calculation_submodule)
    team_module7_workflow.add_node("team_normalization", team_normalization_submodule)
    team_module7_workflow.add_node("team_comment_generation", team_comment_generation_submodule)
    team_module7_workflow.add_node("team_batch_storage", team_batch_storage_submodule)
    
    # 엣지 정의 (순차 실행)
    team_module7_workflow.add_edge(START, "team_data_collection")
    team_module7_workflow.add_edge("team_data_collection", "team_statistics")
    team_module7_workflow.add_edge("team_statistics", "team_score_calculation")
    team_module7_workflow.add_edge("team_score_calculation", "team_normalization")
    team_module7_workflow.add_edge("team_normalization", "team_comment_generation")
    team_module7_workflow.add_edge("team_comment_generation", "team_batch_storage")
    team_module7_workflow.add_edge("team_batch_storage", END)
    
    return team_module7_workflow.compile()

# ================================================================
# 실행 함수들
# ================================================================

def run_team_module7_evaluation(team_id: str, period_id: int = 4):
    """팀 단위 모듈 7 연말 종합평가 실행 (달성률 기반 + CL 정규화 포함)"""
    
    print(f"🚀 팀 단위 모듈 7 + 달성률 기반 CL 정규화 실행 시작: {team_id} (period_id: {period_id})")
    
    # State 정의
    state = TeamModule7AgentState(
        messages=[HumanMessage(content=f"팀 {team_id}: 종합평가 + CL 정규화 시작")],
        team_id=team_id,
        period_id=period_id,
        team_members=[],
        team_achievement_data=[],
        team_fourp_data=[],
        team_quarterly_data={},
        achievement_stats={},
        weights_by_cl={},
        individual_scores=[],
        evaluation_comments=[],
        processed_count=0,
        failed_members=[]
    )
    
    # 그래프 생성 및 실행
    team_module7_graph = create_team_module7_graph()
    
    try:
        result = team_module7_graph.invoke(state)
        
        print("✅ 팀 단위 모듈 7 + 달성률 기반 CL 정규화 실행 완료!")
        print(f"📊 결과:")
        for message in result['messages']:
            print(f"  - {message.content}")
        
        if result.get('processed_count'):
            print(f"🎯 처리 완료: {result['processed_count']}명")
            if result.get('failed_members'):
                print(f"❌ 실패한 팀원: {result['failed_members']}")
        
        return result
        
    except Exception as e:
        print(f"❌ 팀 단위 모듈 7 + 달성률 기반 CL 정규화 실행 실패: {e}")
        return None

def run_multiple_teams_module7(team_ids: List[str], period_id: int = 4):
    """여러 팀 일괄 실행"""
    print(f"🚀 다중 팀 모듈 7 + 달성률 기반 CL 정규화 실행: {len(team_ids)}개 팀")
    
    results = {}
    total_processed = 0
    total_failed = 0
    
    for team_id in team_ids:
        print(f"\n{'='*50}")
        print(f"팀 {team_id} 처리 중...")
        
        result = run_team_module7_evaluation(team_id, period_id)
        results[team_id] = result
        
        if result:
            total_processed += result.get('processed_count', 0)
            total_failed += len(result.get('failed_members', []))
    
    print(f"\n🎯 전체 결과:")
    print(f"   처리된 팀: {len([r for r in results.values() if r is not None])}/{len(team_ids)}")
    print(f"   처리된 인원: {total_processed}명")
    print(f"   실패한 인원: {total_failed}명")
    
    return results

# ================================================================
# 개별 실행 호환 함수 (기존 코드 호환성)
# ================================================================

def run_module7_evaluation(emp_no: str, period_id: int = 4):
    """개별 실행 (기존 호환성을 위한 래퍼 함수)"""
    # 직원의 팀 ID 조회
    with engine.connect() as connection:
        query = text("SELECT team_id FROM employees WHERE emp_no = :emp_no")
        result = connection.execute(query, {"emp_no": emp_no}).fetchone()
        
        if not result:
            print(f"❌ 직원 정보 없음: {emp_no}")
            return None
        
        team_id = result.team_id
    
    print(f"🔄 개별 실행을 팀 단위로 변환: {emp_no} → 팀 {team_id}")
    
    # 팀 단위로 실행
    return run_team_module7_evaluation(team_id, period_id)

# ================================================================
# 테스트 및 디버깅 함수들
# ================================================================

def get_all_teams_with_data(period_id: int = 4) -> List[str]:
    """평가 데이터가 있는 모든 팀 ID 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT DISTINCT e.team_id
            FROM employees e
            JOIN final_evaluation_reports fer ON e.emp_no = fer.emp_no
            JOIN team_evaluations te ON fer.team_evaluation_id = te.team_evaluation_id
            WHERE te.period_id = :period_id
            ORDER BY e.team_id
        """)
        results = connection.execute(query, {"period_id": period_id}).fetchall()
        return [row.team_id for row in results]

def test_team_module7(team_id: str = None, period_id: int = 4):
    """팀 모듈 7 + 달성률 기반 CL 정규화 테스트"""
    if not team_id:
        teams = get_all_teams_with_data(period_id)
        if teams:
            team_id = teams[0]
            print(f"🧪 테스트 팀 자동 선택: {team_id}")
        else:
            print("❌ 테스트할 팀이 없습니다")
            return
    
    return run_team_module7_evaluation(team_id, period_id)

# ================================================================
# 실행 예시
# ================================================================

if __name__ == "__main__":
    print("🚀 팀 단위 모듈 7 + 달성률 기반 CL별 정규화 준비 완료!")
    print("\n주요 변경사항:")
    print("✅ 기여도 → 달성률 변경 (ai_annual_achievement_rate 사용)")
    print("✅ CL별 정규화: 4명 이상일 때만 적용, 3명 이하는 원시점수 유지")
    print("✅ DB 저장: raw_score(원시점수) + score(정규화점수) 별도 저장")
    print("✅ 달성률:4P 가중치 유지 (CL3:6:4, CL2:5:5, CL1:4:6)")
    
    print("\n실행 명령어:")
    print("1. run_team_module7_evaluation('TEAM001', 4)     # 단일 팀 실행")
    print("2. run_multiple_teams_module7(['TEAM001', 'TEAM002'], 4)  # 다중 팀 실행")
    print("3. run_module7_evaluation('E002', 4)             # 개별 실행 (호환성)")
    print("4. test_team_module7()                           # 테스트 실행")
    
    # 자동 테스트 (필요시 주석 해제)
    test_team_module7()

LLM Client initialized: gpt-4o-mini
🚀 팀 단위 모듈 7 + 달성률 기반 CL별 정규화 준비 완료!

주요 변경사항:
✅ 기여도 → 달성률 변경 (ai_annual_achievement_rate 사용)
✅ CL별 정규화: 4명 이상일 때만 적용, 3명 이하는 원시점수 유지
✅ DB 저장: raw_score(원시점수) + score(정규화점수) 별도 저장
✅ 달성률:4P 가중치 유지 (CL3:6:4, CL2:5:5, CL1:4:6)

실행 명령어:
1. run_team_module7_evaluation('TEAM001', 4)     # 단일 팀 실행
2. run_multiple_teams_module7(['TEAM001', 'TEAM002'], 4)  # 다중 팀 실행
3. run_module7_evaluation('E002', 4)             # 개별 실행 (호환성)
4. test_team_module7()                           # 테스트 실행
🧪 테스트 팀 자동 선택: 1
🚀 팀 단위 모듈 7 + 달성률 기반 CL 정규화 실행 시작: 1 (period_id: 4)
🔍 팀 데이터 수집 시작: 1
   팀원 수: 4명
   달성률 데이터: 3건
   4P 데이터: 3건
   분기별 데이터: 3명
📊 달성률 통계: 평균 90.7%, 표준편차 6.4%
🧮 개별 하이브리드 점수 계산 시작...
   E002: 4.55점 (달성률 4.42, 4P 4.75)
   E003: 3.72점 (달성률 3.04, 4P 4.75)
   E004: 3.42점 (달성률 3.04, 4P 4.0)
🔄 팀 내 CL별 정규화 시작...
   CL별 분포: CL3(3명), CL2(0명), CL1(0명)

📊 CL3 정규화 처리:
   CL3 그룹 (3명) 정규화 처리:
     E002: 4.55점 (원시점수 유지)
     E003: 3.72점 (원시점수 유지)
     E004: 3.42점 (원시점수 유지)

📈 정규화 결과

In [3]:
# ================================================================
# 모듈 7: 종합평가 점수 산정 모듈 + CL별 정규화 (연말 팀 단위 처리) - SK 등급 기반 절대평가
# ================================================================

from typing import Annotated, List, Literal, TypedDict, Dict, Optional
from langchain_core.messages import HumanMessage 
import operator
from langgraph.graph import StateGraph, START, END
import json
import re
import sys
import os
import statistics

# 기존 imports
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Row
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, AIMessage
from dotenv import load_dotenv

load_dotenv()

# DB 설정
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../../..')))
from config.settings import DatabaseConfig

db_config = DatabaseConfig()
DATABASE_URL = db_config.DATABASE_URL
engine = create_engine(DATABASE_URL, pool_pre_ping=True)

# LLM 클라이언트 설정
llm_client = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print(f"LLM Client initialized: {llm_client.model_name}")

def row_to_dict(row: Row) -> Dict:
    """SQLAlchemy Row 객체를 딕셔너리로 변환"""
    if row is None:
        return {}
    return row._asdict()

def _extract_json_from_llm_response(text: str) -> str:
    """LLM 응답에서 JSON 블록 추출"""
    match = re.search(r"```(?:json)?\\s*(.*?)\\s*```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

# ================================================================
# TeamModule7AgentState 정의 - 팀 단위 처리
# ================================================================

class TeamModule7AgentState(TypedDict):
    """모듈 7 (종합평가 점수 산정) 상태 - 팀 단위 처리"""
    messages: Annotated[List[HumanMessage], operator.add]
    
    # 팀 기본 정보
    team_id: str
    period_id: int  # 연말: 4
    
    # 팀 전체 데이터 (한 번에 조회)
    team_members: List[Dict]  # 팀원 기본 정보
    team_achievement_data: List[Dict]  # 팀 전체 달성률 데이터
    team_fourp_data: List[Dict]  # 팀 전체 4P 데이터
    team_quarterly_data: Dict  # 팀원별 분기 데이터
    
    # 통계 계산 결과 (팀 공통)
    weights_by_cl: Dict  # CL별 가중치
    
    # 개별 계산 결과
    individual_scores: List[Dict]  # 각 팀원별 점수
    evaluation_comments: List[Dict]  # 각 팀원별 코멘트
    
    # 처리 결과
    processed_count: int
    failed_members: List[str]

# ================================================================
# 가중치 계산 함수
# ================================================================

def get_evaluation_weights_by_cl(cl) -> Dict[str, float]:
    """CL별 디폴트 가중치 반환"""
    # CL 값을 문자열로 변환 (숫자든 문자열이든 처리)
    if isinstance(cl, (int, float)):
        cl_key = f"CL{int(cl)}"
    else:
        cl_key = str(cl).upper()
        if not cl_key.startswith("CL"):
            cl_key = f"CL{cl_key}"
    
    cl_weights = {
        "CL3": {"achievement": 0.6, "fourp": 0.4},
        "CL2": {"achievement": 0.5, "fourp": 0.5}, 
        "CL1": {"achievement": 0.4, "fourp": 0.6}
    }
    return cl_weights.get(cl_key, {"achievement": 0.5, "fourp": 0.5})  # 기본값

# ================================================================
# SK 등급 기반 절대평가 달성률 점수 계산 함수
# ================================================================

def calculate_achievement_score_by_grade(achievement_rate: float) -> tuple[float, str]:
    """SK 등급 체계 기반 절대평가 점수 계산"""
    
    if achievement_rate >= 120:
        # S등급 상한 (120% 이상) → 5.0점
        score = 5.0
        grade = "S+"
        reason = f"달성률 {achievement_rate:.1f}% (S+등급, 탁월한 성과)"
        
    elif achievement_rate >= 110:
        # S등급 (110-120%) → 4.0-5.0점 선형 배치
        progress = (achievement_rate - 110) / 10  # 0~1 사이 값
        score = 4.0 + (progress * 1.0)  # 4.0 ~ 5.0
        grade = "S"
        reason = f"달성률 {achievement_rate:.1f}% (S등급, 매우 우수한 성과)"
        
    elif achievement_rate >= 100:
        # A등급 (100-110%) → 3.5-4.0점 선형 배치  
        progress = (achievement_rate - 100) / 10  # 0~1 사이 값
        score = 3.5 + (progress * 0.5)  # 3.5 ~ 4.0
        grade = "A"
        reason = f"달성률 {achievement_rate:.1f}% (A등급, 목표 달성)"
        
    elif achievement_rate >= 80:
        # B등급 (80-100%) → 2.5-3.5점 선형 배치
        progress = (achievement_rate - 80) / 20  # 0~1 사이 값  
        score = 2.5 + (progress * 1.0)  # 2.5 ~ 3.5
        grade = "B"
        reason = f"달성률 {achievement_rate:.1f}% (B등급, 목표 근접)"
        
    elif achievement_rate >= 60:
        # C등급 (60-80%) → 1.5-2.5점 선형 배치
        progress = (achievement_rate - 60) / 20  # 0~1 사이 값
        score = 1.5 + (progress * 1.0)  # 1.5 ~ 2.5
        grade = "C"
        reason = f"달성률 {achievement_rate:.1f}% (C등급, 목표 미달)"
        
    else:
        # D등급 (60% 미만) → 1.0-1.5점 선형 배치
        if achievement_rate <= 0:
            score = 1.0
        else:
            progress = achievement_rate / 60  # 0~1 사이 값
            score = 1.0 + (progress * 0.5)  # 1.0 ~ 1.5
        grade = "D"
        reason = f"달성률 {achievement_rate:.1f}% (D등급, 크게 미달)"
    
    return round(score, 2), reason

# ================================================================
# CL별 정규화 관련 함수들
# ================================================================

def get_cl_normalization_params(cl) -> Dict[str, float]:
    """CL별 정규화 파라미터 반환 (SK 표준)"""
    # CL 값을 문자열로 변환 (숫자든 문자열이든 처리)
    if isinstance(cl, (int, float)):
        cl_key = f"CL{int(cl)}"
    else:
        cl_key = str(cl).upper()
        if not cl_key.startswith("CL"):
            cl_key = f"CL{cl_key}"
    
    params = {
        "CL3": {"target_mean": 3.5, "target_stdev": 1.7},
        "CL2": {"target_mean": 3.5, "target_stdev": 1.5}, 
        "CL1": {"target_mean": 3.5, "target_stdev": 1.4}
    }
    return params.get(cl_key, {"target_mean": 3.5, "target_stdev": 1.5})

def normalize_cl_group(members: List[Dict], cl: str) -> List[Dict]:
    """CL 그룹 내 정규화 실행 (4명 이상일 때만 적용)"""
    
    if len(members) == 0:
        return members
    
    print(f"   {cl} 그룹 ({len(members)}명) 정규화 처리:")
    
    # 3명 이하인 경우 원시점수 유지
    if len(members) <= 3:
        for member in members:
            member["normalized_score"] = member["hybrid_score"]  # 원시점수 그대로
            member["normalization_reason"] = f"팀 내 {cl} {len(members)}명 (원시점수 유지)"
            member["raw_hybrid_score"] = member["hybrid_score"]  # 원시점수 보관
            print(f"     {member['emp_no']}: {member['hybrid_score']:.2f}점 (원시점수 유지)")
        return members
    
    # 4명 이상인 경우 정규화 적용
    # CL별 목표 파라미터
    params = get_cl_normalization_params(cl)
    target_mean = params["target_mean"]
    target_stdev = params["target_stdev"]
    
    # 원시점수 수집 (하이브리드 점수)
    raw_scores = [m["hybrid_score"] for m in members]
    
    # 현재 통계
    current_mean = statistics.mean(raw_scores)
    current_stdev = statistics.stdev(raw_scores)
    
    print(f"     정규화 적용: 평균 {current_mean:.2f} → {target_mean}, 표준편차 {current_stdev:.2f} → {target_stdev}")
    
    # 정규화 적용
    for member in members:
        raw_score = member["hybrid_score"]
        
        if current_stdev == 0:
            # 모든 점수가 동일한 경우
            normalized_score = target_mean
            reason = f"{cl} 동일점수 → 평균 {target_mean}점"
        else:
            # Z-score 계산 후 목표 분포로 변환
            z_score = (raw_score - current_mean) / current_stdev
            normalized_score = target_mean + (z_score * target_stdev)
            
            # 0.0-5.0 범위 제한 (SK 기준)
            normalized_score = max(0.0, min(5.0, normalized_score))
            
            reason = f"{cl} 정규화 (Z-Score: {z_score:.2f})"
        
        member["normalized_score"] = round(normalized_score, 2)
        member["normalization_reason"] = reason
        member["raw_hybrid_score"] = raw_score  # 원시점수 보관
        
        print(f"     {member['emp_no']}: {raw_score:.2f} → {normalized_score:.2f} ({reason})")
    
    return members

# ================================================================
# 팀 단위 DB 조회 함수들
# ================================================================

def fetch_team_members(team_id: str) -> List[Dict]:
    """팀원 기본 정보 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT emp_no, emp_name, cl, position, team_id
            FROM employees 
            WHERE team_id = :team_id
            ORDER BY cl DESC, position DESC
        """)
        results = connection.execute(query, {"team_id": team_id}).fetchall()
        return [row_to_dict(row) for row in results]

def fetch_team_achievement_data(team_id: str, period_id: int) -> List[Dict]:
    """팀 전체 달성률 데이터 한 번에 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT fer.emp_no, fer.contribution_rate, 
                   fer.ai_annual_achievement_rate,
                   fer.ai_annual_performance_summary_comment, 
                   fer.ai_peer_talk_summary,
                   e.emp_name, e.cl, e.position
            FROM final_evaluation_reports fer
            JOIN team_evaluations te ON fer.team_evaluation_id = te.team_evaluation_id
            JOIN employees e ON fer.emp_no = e.emp_no
            WHERE e.team_id = :team_id AND te.period_id = :period_id
            ORDER BY fer.ai_annual_achievement_rate DESC
        """)
        results = connection.execute(query, {"team_id": team_id, "period_id": period_id}).fetchall()
        return [row_to_dict(row) for row in results]

def fetch_team_fourp_data(team_id: str, period_id: int) -> List[Dict]:
    """팀 전체 4P 데이터 한 번에 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT fer.emp_no, fer.ai_4p_evaluation, e.emp_name
            FROM final_evaluation_reports fer
            JOIN team_evaluations te ON fer.team_evaluation_id = te.team_evaluation_id
            JOIN employees e ON fer.emp_no = e.emp_no
            WHERE e.team_id = :team_id AND te.period_id = :period_id
        """)
        results = connection.execute(query, {"team_id": team_id, "period_id": period_id}).fetchall()
        
        fourp_data = []
        for row in results:
            try:
                fourp_evaluation = json.loads(row.ai_4p_evaluation) if row.ai_4p_evaluation else {}
                fourp_data.append({
                    "emp_no": row.emp_no,
                    "emp_name": row.emp_name,
                    "fourp_results": fourp_evaluation
                })
            except json.JSONDecodeError:
                print(f"4P JSON 파싱 실패: {row.emp_no}")
                fourp_data.append({
                    "emp_no": row.emp_no,
                    "emp_name": row.emp_name,
                    "fourp_results": {}
                })
        
        return fourp_data

def fetch_team_quarterly_data(team_id: str, period_id: int) -> Dict:
    """팀 전체 분기별 Task 데이터 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT t.emp_no, ts.period_id, ts.task_id, t.task_name,
                   ts.ai_contribution_score, ts.ai_analysis_comment_task, ts.task_performance
            FROM task_summaries ts
            JOIN tasks t ON ts.task_id = t.task_id
            JOIN employees e ON t.emp_no = e.emp_no
            WHERE e.team_id = :team_id AND ts.period_id <= :period_id
            ORDER BY t.emp_no, ts.period_id, ts.task_id
        """)
        results = connection.execute(query, {"team_id": team_id, "period_id": period_id}).fetchall()
        
        # emp_no별로 그룹화
        quarterly_data = {}
        for row in results:
            emp_no = row.emp_no
            if emp_no not in quarterly_data:
                quarterly_data[emp_no] = []
            quarterly_data[emp_no].append(row_to_dict(row))
        
        return quarterly_data

def batch_update_temp_evaluations(score_data: List[Dict]) -> Dict:
    """팀 전체 temp_evaluations 배치 업데이트 (raw_score, score 모두 저장)"""
    success_count = 0
    failed_members = []
    
    with engine.connect() as connection:
        try:
            for data in score_data:
                try:
                    raw_score_val = data["raw_score"]
                    try:
                        # raw_score가 json 문자열일 경우 dict로 파싱, 아니면 그대로 사용
                        raw_score_dict = json.loads(raw_score_val)
                        display_raw_score = raw_score_dict.get('raw_hybrid_score', raw_score_val)
                    except (json.JSONDecodeError, TypeError):
                        display_raw_score = raw_score_val

                    query = text("""
                        UPDATE temp_evaluations 
                        SET ai_reason = :ai_reason,
                            raw_score = :raw_score,
                            score = :score,
                            comment = :comment
                        WHERE TempEvaluation_empNo = :emp_no
                    """)
                    
                    result = connection.execute(query, {
                        "emp_no": data["emp_no"],
                        "ai_reason": data["ai_reason"],
                        "raw_score": raw_score_val,  # JSON 문자열 저장
                        "score": data["score"],  # 정규화된 점수 저장
                        "comment": data["comment"]
                    })
                    
                    if result.rowcount > 0:
                        success_count += 1
                        print(f"DB 업데이트 성공: {data['emp_no']} (원시: {display_raw_score}, 정규화: {data['score']})")
                    else:
                        failed_members.append(data["emp_no"])
                        print(f"DB 업데이트 실패: {data['emp_no']} (행 없음)")
                        
                except Exception as e:
                    failed_members.append(data["emp_no"])
                    print(f"DB 업데이트 실패: {data['emp_no']} - {e}")
            
            connection.commit()
            print(f"배치 업데이트 완료: 성공 {success_count}건, 실패 {len(failed_members)}건")
            
            return {
                "success_count": success_count,
                "failed_members": failed_members
            }
            
        except Exception as e:
            print(f"배치 업데이트 실패: {e}")
            connection.rollback()
            return {
                "success_count": 0,
                "failed_members": [data["emp_no"] for data in score_data]
            }

# ================================================================
# LLM 호출 함수
# ================================================================

def call_llm_for_normalized_evaluation_comments(
    emp_data: Dict,
    normalized_score: float,
    raw_hybrid_score: float,
    achievement_score: float,
    fourp_score: float,
    quarterly_tasks: List[Dict],
    fourp_results: Dict,
    achievement_reason: str,
    normalization_reason: str
) -> Dict:
    """정규화된 점수로 ai_reason과 comment 생성"""
    
    emp_no = emp_data["emp_no"]
    emp_name = emp_data.get("emp_name", emp_no)
    position = emp_data.get("position", "직책 정보 없음")
    cl = emp_data.get("cl", "CL 정보 없음")
    
    print(f"LLM Call: {emp_no} 정규화 후 종합평가 근거 생성")
    
    # 분기별 Task 요약
    quarterly_summary = ""
    for i, task in enumerate(quarterly_tasks[:8]):  # 최대 8개만 표시
        quarterly_summary += f"Q{task.get('period_id')}: {task.get('task_name')} - 달성률 {task.get('ai_contribution_score', 0)}점\\n"
        if task.get('ai_analysis_comment_task'):
            quarterly_summary += f"  → {task.get('ai_analysis_comment_task')}\\n"
    
    if not quarterly_summary:
        quarterly_summary = "분기별 Task 데이터 없음"
    
    # 4P 요약
    fourp_summary = f"""
    - Passionate: {fourp_results.get('passionate', {}).get('score', 3.0)}점
    - Proactive: {fourp_results.get('proactive', {}).get('score', 3.0)}점  
    - Professional: {fourp_results.get('professional', {}).get('score', 3.0)}점
    - People: {fourp_results.get('people', {}).get('score', 3.0)}점
    - 평균: {fourp_results.get('overall', {}).get('average_score', 3.0)}점
    """
    
    system_prompt = """
    당신은 SK 조직의 종합 평가 전문가입니다.
    SK 등급 체계 기반 절대평가(달성률)와 정성평가(4P BARS)를 종합하고 CL별 정규화를 거친 최종 점수를 바탕으로 
    두 가지 관점의 평가 근거를 생성해주세요.

    SK 등급 체계 (절대평가):
    - S+등급(120% 이상): 5.0점 (탁월한 성과)
    - S등급(110-120%): 4.0-5.0점 (매우 우수한 성과)
    - A등급(100-110%): 3.5-4.0점 (목표 달성)
    - B등급(80-100%): 2.5-3.5점 (목표 근접)
    - C등급(60-80%): 1.5-2.5점 (목표 미달)
    - D등급(60% 미만): 1.0-1.5점 (크게 미달)

    두 가지 관점의 근거를 생성하세요:
    1. ai_reason: 팀장이 보는 AI 평가 근거 (객관적, 분석적, 절대평가+CL별 정규화 과정 포함)
    2. comment: 팀원이 보는 평가 근거 초안 (개인 친화적, 발전지향적, 격려 포함)

    결과는 JSON 형식으로만 응답하세요.
    """
    
    human_prompt = f"""
    <직원 정보>
    이름: {emp_name}
    사번: {emp_no}
    직책: {position}
    CL: {cl}
    </직원 정보>

    <점수 산출 과정>
    최종 정규화 점수: {normalized_score}점
    ├── 원시 하이브리드 점수: {raw_hybrid_score:.2f}점
    │   ├── 달성률 점수: {achievement_score}점 ({achievement_reason})
    │   └── 4P 점수: {fourp_score}점
    └── CL별 정규화: {normalization_reason}
    </점수 산출 과정>

    <상세 근거 데이터>
    연간 달성률: {emp_data.get('ai_annual_achievement_rate', 0)}%
    성과 요약: {emp_data.get('ai_annual_performance_summary_comment', '성과 요약 없음')}
    동료평가: {emp_data.get('ai_peer_talk_summary', '동료평가 없음')}
    
    4P 평가 결과:
    {fourp_summary}
    
    분기별 Task 성과:
    {quarterly_summary}
    </상세 근거 데이터>

    JSON 응답:
    {{
        "ai_reason": "[팀장용: {emp_name}({emp_no})님에 대한 객관적이고 구체적인 AI 평가 근거. SK 등급 체계 기반 절대평가 결과와 CL별 정규화 과정을 포함하여 분석적 관점으로 설명]",
        "comment": "[팀원용: {emp_name}님께 드리는 개인 친화적이고 발전지향적인 평가 근거 초안. 최종 점수 {normalized_score}점에 대한 격려와 성장 방향 포함]"
    }}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_prompt),
        HumanMessage(content=human_prompt)
    ])
    
    chain = prompt | llm_client

    try:
        response: AIMessage = chain.invoke({})
        json_output_raw = response.content
        json_output = _extract_json_from_llm_response(json_output_raw)
        llm_parsed_data = json.loads(json_output)
        
        ai_reason = llm_parsed_data.get("ai_reason", "")
        comment = llm_parsed_data.get("comment", "")

        if not ai_reason or not comment:
            raise ValueError("LLM 응답에서 ai_reason 또는 comment가 누락됨")

        return {
            "ai_reason": ai_reason,
            "comment": comment
        }

    except json.JSONDecodeError as e:
        print(f"LLM 응답 JSON 파싱 오류: {e}")
        return {
            "ai_reason": f"{emp_name}님의 정규화 후 종합 평가 근거 생성 중 오류 발생",
            "comment": f"{emp_name}님께 드리는 정규화 후 평가 근거 생성 중 오류 발생"
        }
    except Exception as e:
        print(f"LLM 호출 중 오류: {e}")
        return {
            "ai_reason": f"{emp_name}님의 정규화 후 종합 평가 근거 생성 중 오류 발생",
            "comment": f"{emp_name}님께 드리는 정규화 후 평가 근거 생성 중 오류 발생"
        }

# ================================================================
# 팀 단위 서브모듈 함수들
# ================================================================

def team_data_collection_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """1. 팀 데이터 수집 서브모듈"""
    
    team_id = state["team_id"]
    period_id = state["period_id"]
    
    try:
        print(f"🔍 팀 데이터 수집 시작: {team_id}")
        
        # 1. 팀원 기본 정보 조회
        team_members = fetch_team_members(team_id)
        print(f"   팀원 수: {len(team_members)}명")
        
        # 2. 팀 전체 달성률 데이터 조회
        team_achievement_data = fetch_team_achievement_data(team_id, period_id)
        print(f"   달성률 데이터: {len(team_achievement_data)}건")
        
        # 3. 팀 전체 4P 데이터 조회
        team_fourp_data = fetch_team_fourp_data(team_id, period_id)
        print(f"   4P 데이터: {len(team_fourp_data)}건")
        
        # 4. 팀 전체 분기별 데이터 조회
        team_quarterly_data = fetch_team_quarterly_data(team_id, period_id)
        print(f"   분기별 데이터: {len(team_quarterly_data)}명")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"데이터 수집 완료")],
            "team_members": team_members,
            "team_achievement_data": team_achievement_data,
            "team_fourp_data": team_fourp_data,
            "team_quarterly_data": team_quarterly_data
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"데이터 수집 실패: {str(e)}")]
        raise e

def team_weights_calculation_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """2. CL별 가중치 계산 서브모듈"""
    
    try:
        team_members = state["team_members"]
        
        # CL별 가중치 계산
        weights_by_cl = {}
        for member in team_members:
            cl = member.get("cl", "CL2")
            if cl not in weights_by_cl:
                weights_by_cl[cl] = get_evaluation_weights_by_cl(cl)
        
        print(f"📊 CL별 가중치 설정 완료: {list(weights_by_cl.keys())}")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content="가중치 계산 완료")],
            "weights_by_cl": weights_by_cl
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"가중치 계산 실패: {str(e)}")]
        raise e

def team_score_calculation_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """3. 팀 전체 점수 계산 서브모듈 (SK 등급 기반 절대평가)"""
    
    try:
        team_achievement_data = state["team_achievement_data"]
        team_fourp_data = state["team_fourp_data"]
        weights_by_cl = state["weights_by_cl"]
        
        individual_scores = []
        
        print("🧮 SK 등급 기반 절대평가 점수 계산 시작...")
        
        for achievement_data in team_achievement_data:
            emp_no = achievement_data["emp_no"]
            cl = achievement_data.get("cl", "CL2")
            
            # 가중치 조회
            weights = weights_by_cl.get(cl, {"achievement": 0.5, "fourp": 0.5})
            
            # SK 등급 기반 절대평가 달성률 점수 계산
            achievement_score, achievement_reason = calculate_achievement_score_by_grade(
                achievement_data["ai_annual_achievement_rate"]
            )
            
            # 4P 점수 조회
            fourp_data = next((fp for fp in team_fourp_data if fp["emp_no"] == emp_no), {})
            fourp_results = fourp_data.get("fourp_results", {})
            fourp_score = fourp_results.get("overall", {}).get("average_score", 3.0)
            
            # 하이브리드 점수 계산 (정규화 전 원시점수)
            hybrid_score = (achievement_score * weights["achievement"]) + (fourp_score * weights["fourp"])
            final_score = round(hybrid_score, 2)
            
            individual_scores.append({
                "emp_no": emp_no,
                "emp_name": achievement_data.get("emp_name"),
                "cl": cl,
                "achievement_score": achievement_score,
                "achievement_reason": achievement_reason,
                "fourp_score": fourp_score,
                "hybrid_score": final_score,  # 정규화 전 원시점수
                "weights": weights,
                "emp_data": achievement_data,
                "fourp_results": fourp_results
            })
            
            print(f"   {emp_no}: {final_score}점 (달성률 {achievement_score}, 4P {fourp_score}) - {achievement_reason}")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"절대평가 점수 계산 완료: {len(individual_scores)}명")],
            "individual_scores": individual_scores
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"점수 계산 실패: {str(e)}")]
        raise e

def team_normalization_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """4. 팀 내 CL별 정규화 서브모듈"""
    
    try:
        individual_scores = state["individual_scores"]
        
        print("🔄 팀 내 CL별 정규화 시작...")
        
        # 1. CL별 그룹화 (숫자/문자열 모두 처리)
        cl_groups = {
            "CL1": [],
            "CL2": [], 
            "CL3": []
        }
        
        for score in individual_scores:
            cl_raw = score.get("cl", 2)  # 기본값 2
            
            # CL 값 정규화
            if isinstance(cl_raw, (int, float)):
                cl = f"CL{int(cl_raw)}"
            else:
                cl = str(cl_raw).upper()
                if not cl.startswith("CL"):
                    cl = f"CL{cl}"
            
            # 유효한 CL인지 확인
            if cl in cl_groups:
                cl_groups[cl].append(score)
                score["cl"] = cl  # 정규화된 CL 값으로 업데이트
            else:
                print(f"⚠️ 알 수 없는 CL: {cl_raw} → CL2로 처리")
                cl_groups["CL2"].append(score)
                score["cl"] = "CL2"
        
        print(f"   CL별 분포: CL3({len(cl_groups['CL3'])}명), CL2({len(cl_groups['CL2'])}명), CL1({len(cl_groups['CL1'])}명)")
        
        # 2. CL별 정규화 실행 (4명 이상일 때만)
        normalized_scores = []
        
        for cl, members in cl_groups.items():
            if len(members) > 0:
                print(f"\\n📊 {cl} 정규화 처리:")
                normalized_members = normalize_cl_group(members, cl)
                normalized_scores.extend(normalized_members)
        
        # 3. 정규화 통계 출력
        raw_scores = [s["hybrid_score"] for s in individual_scores]
        norm_scores = [s["normalized_score"] for s in normalized_scores]
        
        print(f"\\n📈 정규화 결과:")
        print(f"   원시점수: 평균 {statistics.mean(raw_scores):.2f}, 표준편차 {statistics.stdev(raw_scores) if len(raw_scores) > 1 else 0:.2f}")
        print(f"   정규화점수: 평균 {statistics.mean(norm_scores):.2f}, 표준편차 {statistics.stdev(norm_scores) if len(norm_scores) > 1 else 0:.2f}")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"CL별 정규화 완료: {len(normalized_scores)}명")],
            "individual_scores": normalized_scores
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"정규화 실패: {str(e)}")]
        raise e

def team_comment_generation_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """5. 팀 전체 코멘트 생성 서브모듈 (정규화된 점수 기준)"""
    
    try:
        individual_scores = state["individual_scores"]
        team_quarterly_data = state["team_quarterly_data"]
        
        evaluation_comments = []
        
        print("💬 정규화 후 평가 코멘트 생성 시작...")
        
        for score_data in individual_scores:
            emp_no = score_data["emp_no"]
            quarterly_tasks = team_quarterly_data.get(emp_no, [])
            
            # LLM을 통한 정규화 후 코멘트 생성
            llm_result = call_llm_for_normalized_evaluation_comments(
                emp_data=score_data["emp_data"],
                normalized_score=score_data["normalized_score"],
                raw_hybrid_score=score_data["raw_hybrid_score"],
                achievement_score=score_data["achievement_score"],
                fourp_score=score_data["fourp_score"],
                quarterly_tasks=quarterly_tasks,
                fourp_results=score_data["fourp_results"],
                achievement_reason=score_data["achievement_reason"],
                normalization_reason=score_data["normalization_reason"]
            )
            
            # raw_score에 저장할 JSON 데이터 구성
            fourp_results = score_data.get("fourp_results", {})
            raw_score_details = {
                "achievement_score": score_data.get("achievement_score"),
                "passionate_score": fourp_results.get("passionate", {}).get("score", 3.0),
                "proactive_score": fourp_results.get("proactive", {}).get("score", 3.0),
                "professional_score": fourp_results.get("professional", {}).get("score", 3.0),
                "people_score": fourp_results.get("people", {}).get("score", 3.0),
                "raw_hybrid_score": score_data.get("raw_hybrid_score")
            }
            raw_score_json = json.dumps(raw_score_details, ensure_ascii=False)

            evaluation_comments.append({
                "emp_no": emp_no,
                "emp_name": score_data.get("emp_name"),
                "raw_score": raw_score_json,  # 원시점수를 JSON 문자열로 저장
                "score": score_data["normalized_score"],  # 정규화된 점수
                "ai_reason": llm_result["ai_reason"],
                "comment": llm_result["comment"]
            })
            
            print(f"   {emp_no}: 정규화 후 코멘트 생성 완료")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"코멘트 생성 완료: {len(evaluation_comments)}명")],
            "evaluation_comments": evaluation_comments
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state["messages"] = [HumanMessage(content=f"코멘트 생성 실패: {str(e)}")]
        raise e

def team_batch_storage_submodule(state: TeamModule7AgentState) -> TeamModule7AgentState:
    """6. 팀 배치 저장 서브모듈"""
    
    try:
        evaluation_comments = state["evaluation_comments"]
        
        print("💾 배치 저장 시작...")
        
        # 배치 업데이트 실행
        update_result = batch_update_temp_evaluations(evaluation_comments)
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"배치 저장 완료: 성공 {update_result['success_count']}건")],
            "processed_count": update_result["success_count"],
            "failed_members": update_result["failed_members"]
        })
        return updated_state
        
    except Exception as e:
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"배치 저장 실패: {str(e)}")],
            "processed_count": 0,
            "failed_members": []
        })
        raise e

# ================================================================
# 팀 단위 워크플로우 생성
# ================================================================

def create_team_module7_graph():
    """팀 단위 모듈 7 그래프 생성 및 반환 (SK 등급 기반 절대평가 + CL 정규화 포함)"""
    team_module7_workflow = StateGraph(TeamModule7AgentState)
    
    # 노드 추가
    team_module7_workflow.add_node("team_data_collection", team_data_collection_submodule)
    team_module7_workflow.add_node("team_weights_calculation", team_weights_calculation_submodule)
    team_module7_workflow.add_node("team_score_calculation", team_score_calculation_submodule)
    team_module7_workflow.add_node("team_normalization", team_normalization_submodule)
    team_module7_workflow.add_node("team_comment_generation", team_comment_generation_submodule)
    team_module7_workflow.add_node("team_batch_storage", team_batch_storage_submodule)
    
    # 엣지 정의 (순차 실행)
    team_module7_workflow.add_edge(START, "team_data_collection")
    team_module7_workflow.add_edge("team_data_collection", "team_weights_calculation")
    team_module7_workflow.add_edge("team_weights_calculation", "team_score_calculation")
    team_module7_workflow.add_edge("team_score_calculation", "team_normalization")
    team_module7_workflow.add_edge("team_normalization", "team_comment_generation")
    team_module7_workflow.add_edge("team_comment_generation", "team_batch_storage")
    team_module7_workflow.add_edge("team_batch_storage", END)
    
    return team_module7_workflow.compile()

# ================================================================
# 실행 함수들
# ================================================================

def run_team_module7_evaluation(team_id: str, period_id: int = 4):
    """팀 단위 모듈 7 연말 종합평가 실행 (SK 등급 기반 절대평가 + CL 정규화 포함)"""
    
    print(f"🚀 팀 단위 모듈 7 + SK 등급 기반 절대평가 + CL 정규화 실행 시작: {team_id} (period_id: {period_id})")
    
    # State 정의
    state = TeamModule7AgentState(
        messages=[HumanMessage(content=f"팀 {team_id}: SK 등급 기반 종합평가 + CL 정규화 시작")],
        team_id=team_id,
        period_id=period_id,
        team_members=[],
        team_achievement_data=[],
        team_fourp_data=[],
        team_quarterly_data={},
        weights_by_cl={},
        individual_scores=[],
        evaluation_comments=[],
        processed_count=0,
        failed_members=[]
    )
    
    # 그래프 생성 및 실행
    team_module7_graph = create_team_module7_graph()
    
    try:
        result = team_module7_graph.invoke(state)
        
        print("✅ 팀 단위 모듈 7 + SK 등급 기반 절대평가 + CL 정규화 실행 완료!")
        print(f"📊 결과:")
        for message in result['messages']:
            print(f"  - {message.content}")
        
        if result.get('processed_count'):
            print(f"🎯 처리 완료: {result['processed_count']}명")
            if result.get('failed_members'):
                print(f"❌ 실패한 팀원: {result['failed_members']}")
        
        return result
        
    except Exception as e:
        print(f"❌ 팀 단위 모듈 7 + SK 등급 기반 절대평가 + CL 정규화 실행 실패: {e}")
        return None

def run_multiple_teams_module7(team_ids: List[str], period_id: int = 4):
    """여러 팀 일괄 실행"""
    print(f"🚀 다중 팀 모듈 7 + SK 등급 기반 절대평가 + CL 정규화 실행: {len(team_ids)}개 팀")
    
    results = {}
    total_processed = 0
    total_failed = 0
    
    for team_id in team_ids:
        print(f"\\n{'='*50}")
        print(f"팀 {team_id} 처리 중...")
        
        result = run_team_module7_evaluation(team_id, period_id)
        results[team_id] = result
        
        if result:
            total_processed += result.get('processed_count', 0)
            total_failed += len(result.get('failed_members', []))
    
    print(f"\\n🎯 전체 결과:")
    print(f"   처리된 팀: {len([r for r in results.values() if r is not None])}/{len(team_ids)}")
    print(f"   처리된 인원: {total_processed}명")
    print(f"   실패한 인원: {total_failed}명")
    
    return results

# ================================================================
# 개별 실행 호환 함수 (기존 코드 호환성)
# ================================================================

def run_module7_evaluation(emp_no: str, period_id: int = 4):
    """개별 실행 (기존 호환성을 위한 래퍼 함수)"""
    # 직원의 팀 ID 조회
    with engine.connect() as connection:
        query = text("SELECT team_id FROM employees WHERE emp_no = :emp_no")
        result = connection.execute(query, {"emp_no": emp_no}).fetchone()
        
        if not result:
            print(f"❌ 직원 정보 없음: {emp_no}")
            return None
        
        team_id = result.team_id
    
    print(f"🔄 개별 실행을 팀 단위로 변환: {emp_no} → 팀 {team_id}")
    
    # 팀 단위로 실행
    return run_team_module7_evaluation(team_id, period_id)

# ================================================================
# 테스트 및 디버깅 함수들
# ================================================================

def get_all_teams_with_data(period_id: int = 4) -> List[str]:
    """평가 데이터가 있는 모든 팀 ID 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT DISTINCT e.team_id
            FROM employees e
            JOIN final_evaluation_reports fer ON e.emp_no = fer.emp_no
            JOIN team_evaluations te ON fer.team_evaluation_id = te.team_evaluation_id
            WHERE te.period_id = :period_id
            ORDER BY e.team_id
        """)
        results = connection.execute(query, {"period_id": period_id}).fetchall()
        return [row.team_id for row in results]

def test_team_module7(team_id: str = None, period_id: int = 4):
    """팀 모듈 7 + SK 등급 기반 절대평가 + CL 정규화 테스트"""
    if not team_id:
        teams = get_all_teams_with_data(period_id)
        if teams:
            team_id = teams[0]
            print(f"🧪 테스트 팀 자동 선택: {team_id}")
        else:
            print("❌ 테스트할 팀이 없습니다")
            return
    
    return run_team_module7_evaluation(team_id, period_id)

def preview_achievement_scoring():
    """SK 등급 기반 절대평가 점수 미리보기"""
    print("📋 SK 등급 기반 절대평가 점수 체계:")
    print("=" * 60)
    
    test_rates = [0, 30, 50, 70, 85, 95, 100, 105, 115, 125, 150, 200]
    
    for rate in test_rates:
        score, reason = calculate_achievement_score_by_grade(rate)
        print(f"달성률 {rate:3d}% → {score:4.2f}점 ({reason.split('(')[1].split(',')[0]})")
    
    print("=" * 60)

# ================================================================
# 실행 예시
# ================================================================

if __name__ == "__main__":
    print("🚀 팀 단위 모듈 7 + SK 등급 기반 절대평가 + CL별 정규화 준비 완료!")
    print("\\n🔥 주요 변경사항:")
    print("✅ SK 등급 체계 기반 절대평가 (S/A/B/C/D → 1-5점)")
    print("✅ 달성률 100% = 3.5점, 120% = 5.0점 명확한 기준")
    print("✅ CL별 정규화: 4명 이상일 때만 적용, 3명 이하는 원시점수 유지")
    print("✅ DB 저장: raw_score(원시점수) + score(정규화점수) 별도 저장")
    print("✅ 달성률:4P 가중치 유지 (CL3:6:4, CL2:5:5, CL1:4:6)")
    
    print("\\n실행 명령어:")
    print("1. run_team_module7_evaluation('TEAM001', 4)     # 단일 팀 실행")
    print("2. run_multiple_teams_module7(['TEAM001', 'TEAM002'], 4)  # 다중 팀 실행")
    print("3. run_module7_evaluation('E002', 4)             # 개별 실행 (호환성)")
    print("4. test_team_module7()                           # 테스트 실행")
    print("5. preview_achievement_scoring()                 # 점수 체계 미리보기")
    
    # 점수 체계 미리보기 실행
    preview_achievement_scoring()
    
    # 자동 테스트 (필요시 주석 해제)
    test_team_module7()

LLM Client initialized: gpt-4o-mini
🚀 팀 단위 모듈 7 + SK 등급 기반 절대평가 + CL별 정규화 준비 완료!
\n🔥 주요 변경사항:
✅ SK 등급 체계 기반 절대평가 (S/A/B/C/D → 1-5점)
✅ 달성률 100% = 3.5점, 120% = 5.0점 명확한 기준
✅ CL별 정규화: 4명 이상일 때만 적용, 3명 이하는 원시점수 유지
✅ DB 저장: raw_score(원시점수) + score(정규화점수) 별도 저장
✅ 달성률:4P 가중치 유지 (CL3:6:4, CL2:5:5, CL1:4:6)
\n실행 명령어:
1. run_team_module7_evaluation('TEAM001', 4)     # 단일 팀 실행
2. run_multiple_teams_module7(['TEAM001', 'TEAM002'], 4)  # 다중 팀 실행
3. run_module7_evaluation('E002', 4)             # 개별 실행 (호환성)
4. test_team_module7()                           # 테스트 실행
5. preview_achievement_scoring()                 # 점수 체계 미리보기
📋 SK 등급 기반 절대평가 점수 체계:
달성률   0% → 1.00점 (D등급)
달성률  30% → 1.25점 (D등급)
달성률  50% → 1.42점 (D등급)
달성률  70% → 2.00점 (C등급)
달성률  85% → 2.75점 (B등급)
달성률  95% → 3.25점 (B등급)
달성률 100% → 3.50점 (A등급)
달성률 105% → 3.75점 (A등급)
달성률 115% → 4.50점 (S등급)
달성률 125% → 5.00점 (S+등급)
달성률 150% → 5.00점 (S+등급)
달성률 200% → 5.00점 (S+등급)
🧪 테스트 팀 자동 선택: 1
🚀 팀 단위 모듈 7 + SK 등급 기반 절대평가 + CL 정규화 실행 시작: 1 (period_id: 4)
